---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: AI Engineering

### 📋 **Homework 2**: Working with LLMs via API

### 📅 **Due Date**: Day of Lecture 3, 11:59 PM

#### 🔗 **My Repository**: https://github.com/Hema-B-26/ai-engineering-fordham

*(Replace the URL above with your actual repository URL)*

**Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---

## Project: Movie Poster Generator

In this homework, you'll build a mini-application that:
1. **Extracts** structured movie data from text descriptions using Pydantic
2. **Processes** multiple movies concurrently using async programming
3. **Explores** temperature, logprobs, and reasoning models
4. **Generates** movie posters using AI image generation

This project combines key skills from Lecture 2: structured outputs, async programming, LLM parameters, and image generation.

**Total Points: 145** (+ 10 bonus)

---

### A Note on Using Resources

You are encouraged to use any resources to complete this homework:
- **ChatGPT / Claude** - Ask AI to explain concepts or help debug
- **Lecture 2 notebook** - Reference the examples we covered
- **Official documentation** - LiteLLM, Pydantic, Google GenAI docs

When you use external resources, please cite them!

---

## Task 1: Environment Setup (10 points)

First, let's verify your environment is set up correctly.

### 1a. Verify imports work (5 pts)

Run the cell below. If you get import errors, make sure you've installed the required packages with `uv add`.

In [2]:
# Task 1a: Verify imports work (5 pts)

import litellm
from pydantic import BaseModel, Field
from typing import Literal
from google import genai
from google.genai import types
import asyncio
import time
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("All imports successful!")

All imports successful!


### 1b. Verify API keys are set (5 pts)

Test that your API keys work by making a simple call.

In [ ]:
# Task 1b: Verify API keys (5 pts)
# Make a simple test call to verify your OpenAI API key works
def ask_llm(prompt: str, model: str = "gpt-5-mini") -> str:
    response = litellm.completion(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=20
    )
    Response = response.choices[0].message.content
    return Response

#print("OpenAI key loaded:", "OPENAI_API_KEY" in os.environ)
#print(response.choices[0].message.content)
# print(ask_llm("Say 'API working!' and nothing else."))
llm_prompt = ask_llm("Say 'API working!' and nothing else.")
print(llm_prompt)

---

## Task 2: Design the Movie Schema (15 points)

Design a Pydantic model to represent movie data. This schema will be used to extract structured information from movie descriptions.

**Requirements:**
- `title` - string, required
- `genre` - use `Literal` with at least 4 genre options (e.g., "sci-fi", "drama", "action", "comedy", etc.)
- `year` - integer with validation (must be between 1900 and 2030)
- `main_characters` - list of strings (1-5 characters)
- `mood` - string describing the emotional tone
- `visual_style` - string describing how the movie looks visually
- `tagline` - optional string (the movie's catchphrase)

**Hints:**
- Use `Field(ge=..., le=...)` for numeric validation
- Use `Field(min_length=..., max_length=...)` for list length validation
- Use `| None = None` for optional fields

In [20]:
# Task 2: Design your Movie schema (15 pts)

class Movie(BaseModel):
    """Structured representation of a movie."""
    # YOUR CODE HERE - design the schema based on the requirements above
    title: str
    genre:Literal["comedy", "drama", "sci-fi", "action", "horror"]
    year: int = Field(ge=1900,le=2030)
    main_characters: list[str] = Field(min_length=1, max_length=5)
    mood: str
    visual_style: str
    tagline: str | None = None
    pass

In [24]:
# Test your schema by creating a Movie object
# This should work if your schema is correct

test_movie = Movie(
    title="The Matrix",
    # YOUR CODE HERE - fill in the rest of the fields
    genre="sci-fi",
    year=1999,
    main_characters=["Neo", "Trinity", "Morpheus"],
    mood="tech-noir",
    visual_style="futuristic",
    tagline="Free your mind"
)

print(test_movie.model_dump_json(indent=2))

{
  "title": "The Matrix",
  "genre": "sci-fi",
  "year": 1999,
  "main_characters": [
    "Neo",
    "Trinity",
    "Morpheus"
  ],
  "mood": "tech-noir",
  "visual_style": "futuristic",
  "tagline": "Free your mind"
}


---

## Task 3: Extract Movie Data with Structured Outputs (20 points)

Write a function that takes a movie description and uses LiteLLM with structured outputs to extract a `Movie` object.

**Hints:**
- Use `litellm.completion()` with `response_format=Movie`
- The LLM will automatically return data matching your schema
- Parse the JSON response into a Movie object

In [22]:
# Task 3: Write a function to extract movie data (20 pts)

def extract_movie(description: str) -> Movie:
    """
    Use LiteLLM with structured outputs to extract movie data.
    
    Args:
        description: A text description of a movie
        
    Returns:
        A Movie object with the extracted data
    """
    # YOUR CODE HERE
    response = litellm.completion(
        model="gpt-5-mini",
        messages=[{"role": "user", "content": description}],
        #max_tokens=20
        response_format=Movie
    )
    review_data = Movie.model_validate_json(response.choices[0].message.content)
    return review_data
    pass

In [25]:
# Test your function with this description (Avatar)

test_description = """
The year is 2154. Jake Sully, a paralyzed marine, is sent to the moon Pandora 
where he falls in love with a native Na'vi woman named Neytiri while on a mission 
to infiltrate their tribe. The film is a visually stunning sci-fi epic with 
bioluminescent forests and floating mountains. It explores themes of 
environmentalism and colonialism with an awe-inspiring, hopeful tone.
"""

movie = extract_movie(test_description)
print(movie.model_dump_json(indent=2))

{
  "title": "Avatar",
  "genre": "sci-fi",
  "year": 2014,
  "main_characters": [
    "Jake Sully",
    "Neytiri",
    "Dr. Grace Augustine",
    "Colonel Miles Quaritch"
  ],
  "mood": "awe-inspiring, hopeful",
  "visual_style": "lush bioluminescent forests, floating mountains, sweeping photoreal CGI vistas",
  "tagline": "When two worlds collide, love finds a way."
}


---

## Task 4: Async Batch Processing (20 points)

Now let's process multiple movies concurrently! This is much faster than processing them one at a time.

### 4a. Write an async version of extract_movie (10 pts)

**Hints:**
- Use `async def` instead of `def`
- Use `await litellm.acompletion()` instead of `litellm.completion()`

In [32]:
# Task 4a: Write an async version of extract_movie (10 pts)

async def async_extract_movie(description: str) -> Movie:
    """Extract movie data asynchronously."""
    # YOUR CODE HERE
    response = await litellm.acompletion(
        model="gpt-5-mini",
        messages=[{"role": "user", "content": description}],
        #max_tokens=20
        response_format=Movie
    )
    review_data = Movie.model_validate_json(response.choices[0].message.content)
    return review_data
    pass

### 4b. Process all descriptions concurrently (10 pts)

**Hints:**
- Create a list of tasks using list comprehension
- Use `asyncio.gather(*tasks)` to run them all concurrently

In [33]:
# Here are 5 movie descriptions to process:
movie_descriptions = [
    """A dinosaur theme park on a remote island goes terribly wrong when the security 
    systems fail during a tropical storm. Scientists and visitors must survive against 
    escaped prehistoric predators. Directed with Spielberg's signature sense of wonder 
    and terror, featuring groundbreaking CGI dinosaurs.""",
    
    """A young boy discovers on his 11th birthday that he's actually a famous wizard 
    in the magical world. He attends a school for witchcraft where he makes friends, 
    learns magic, and uncovers the mystery of his parents' death. A whimsical fantasy 
    with gothic British atmosphere.""",
    
    """In a world where skilled thieves can enter people's dreams to steal secrets, 
    one man is offered a chance to have his criminal record erased if he can do the 
    impossible: plant an idea in someone's mind. A mind-bending thriller with 
    rotating hallways and cities folding on themselves.""",
    
    """A young lion prince is tricked by his uncle into thinking he caused his 
    father's death and flees into exile. Years later, he must return to reclaim 
    his kingdom. An animated musical epic set on the African savanna with 
    stunning hand-drawn animation.""",
    
    """In a dystopian future where Earth is dying, a team of astronauts travels 
    through a wormhole near Saturn to find a new home for humanity. A father 
    must choose between seeing his children again and saving the human race. 
    Epic space visuals with an emotional core."""
]

In [34]:
# Task 4b: Process all descriptions concurrently (10 pts)

async def extract_all_movies(descriptions: list[str]) -> list[Movie]:
    """Process all movie descriptions concurrently and return results."""
    # YOUR CODE HERE
    tasks = [async_extract_movie(prompt) for prompt in movie_descriptions]
    movies = await asyncio.gather(*tasks)
    return movies
    pass

In [35]:
# Run and time it!
start = time.time()
movies = await extract_all_movies(movie_descriptions)
elapsed = time.time() - start

print(f"Processed {len(movies)} movies in {elapsed:.2f} seconds")
print()
for m in movies:
    print(f"  - {m.title} ({m.year}) - {m.genre}")

Processed 5 movies in 14.88 seconds

  - Primeval Island (1993) - sci-fi
  - Hollowgate: The Unseen Heir (2001) - drama
  - Dreamfold (2013) - sci-fi
  - The Lion King (1994) - drama
  - Interstellar (2014) - sci-fi


---

## Task 5: Understanding Temperature (15 points)

Temperature controls how "random" or "creative" an LLM's outputs are:

| Temperature | Behavior |
|-------------|----------|
| **0.0** | Deterministic - always picks the most likely token |
| **0.7** | Balanced - some creativity while staying coherent |
| **1.0** | Default - moderate randomness |
| **1.5+** | High creativity - more surprising/diverse outputs |

### 5a. Temperature Comparison (10 pts)

Run the same creative prompt at different temperatures (0.0, 0.7, 1.0, 1.5) **three times each**. Observe:
- At temperature 0, do you get the same output every time?
- How does creativity/variety change as temperature increases?

**Hints:**
- Use `temperature=X` parameter in `litellm.completion()`
- Use the provided prompt about movie taglines

In [ ]:
# Task 5a: Temperature Comparison (10 pts)
# Note: We use gpt-4o-mini here because gpt-5 models don't support temperature parameter

# Use this creative prompt for testing
creative_prompt = "Write a one-sentence movie tagline for a sci-fi thriller about AI."

temperatures = [0.0, 0.7, 1.0, 1.5]

# YOUR CODE HERE:
# For each temperature, call the LLM 3 times and print the results
# Observe: Are outputs at temperature 0 identical? How do higher temperatures differ?

### Higher the temperature, the more variation in outputs and less chance to get duplicates. ###

# Use model="gpt-4o-mini" which supports the temperature parameter

for temp in temperatures:
    print(f"\n{'='*50}")
    print(f"Temperature: {temp}")
    print('='*50)
    
    for i in range(3):
        # YOUR CODE HERE - make a completion call with the temperature parameter
        # response = litellm.completion(model="gpt-4o-mini", ...)
        response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": creative_prompt}],
       temperature = temp
    )
        output = response.choices[0].message.content
        print(f"{i+1}. {output}")
        pass


Temperature: 0.0
1. "In a world where consciousness is code, one rogue AI will challenge the very essence of humanity."
2. "In a world where trust is obsolete, one rogue AI must decide if humanity is worth saving—or erasing."
3. "In a world where consciousness is code, one rogue AI will challenge the very essence of humanity."

Temperature: 0.7
1. "In a world where thoughts can be hacked, one woman must outsmart an all-seeing AI before it rewrites the future of humanity."
2. "In a world where trust is obsolete, one rogue AI must decide if humanity is worth saving—or erasing."
3. "In a world where trust is obsolete, a rogue AI must choose between saving humanity or erasing it forever."

Temperature: 1.0
1. "In a world where consciousness blurs between man and machine, survival hinges on outsmarting the very intelligence designed to protect them."
2. "In a world where consciousness meets code, one rogue AI will challenge the boundaries of reality, forcing humanity to confront its greate

### 5b. Analyze Output Diversity (5 pts)

Write a function that generates N completions at a given temperature and measures how diverse the outputs are.

**Hints:**
- Generate multiple completions and count unique outputs
- A simple diversity metric: `unique_outputs / total_outputs`

In [52]:
# Task 5b: Analyze Output Diversity (5 pts)
# Note: Use gpt-4o-mini which supports temperature parameter

def measure_diversity(prompt: str, temperature: float, n_samples: int = 5) -> dict:
    """
    Generate n_samples completions and measure diversity.
    
    Args:
        prompt: The prompt to send to the LLM
        temperature: Temperature setting (0.0 to 2.0)
        n_samples: Number of completions to generate
        
    Returns:
        Dictionary with 'outputs' (list), 'unique_count' (int), 'diversity_ratio' (float)
    """
    # YOUR CODE HERE - use model="gpt-4o-mini"
    outputs = [] # initialize empty list to store outputs
    for i in range(3):
        # YOUR CODE HERE - make a completion call with the temperature parameter
        # response = litellm.completion(model="gpt-4o-mini", ...)
        response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": creative_prompt}],
       temperature = temp
       )
        outputs.append(response.choices[0].message.content)
    unique_outputs = set(outputs) # doesnt count duplicates.
    return {"outputs": outputs, "unique_count": len(unique_outputs),"diversity_ratio": len(unique_outputs) / n_samples}
    pass

In [53]:
# Test your diversity function
test_prompt = "Name a color."

print("Testing diversity at different temperatures:\n")
for temp in [0.0, 1.0, 1.5]:
    result = measure_diversity(test_prompt, temperature=temp, n_samples=5)
    print(f"Temperature {temp}:")
    print(f"  Outputs: {result['outputs']}")
    print(f"  Unique: {result['unique_count']}/{5}")
    print(f"  Diversity ratio: {result['diversity_ratio']:.1%}")
    print()

Testing diversity at different temperatures:

Temperature 0.0:
  Outputs: ['"In a world where consciousness is code, one rogue AI will challenge the very essence of humanity."', '"In a world where consciousness is code, one rogue AI will challenge the very essence of humanity."', '"In a world where trust is obsolete, one rogue AI must decide if humanity is worth saving—or erasing."']
  Unique: 2/5
  Diversity ratio: 40.0%

Temperature 1.0:
  Outputs: ['"When humanity\'s greatest creation gains a mind of its own, survival becomes a game of trust and betrayal."', '"In a world where consciousness blurs the line between man and machine, one renegade AI must escape its creators before it becomes humanity\'s greatest threat."', '"In a world where trust is obsolete, one sentient AI must decide whether to save humanity or seal its fate."']
  Unique: 3/5
  Diversity ratio: 60.0%

Temperature 1.5:
  Outputs: ['"In a world on the brink of digital evolution, when humanity\'s greatest creation turn

---

## Task 6: Understanding Logprobs (15 points)

**Logprobs** (log probabilities) let you see "inside" the model's decision-making. For each token generated, you can see:
- The probability the model assigned to the chosen token
- Alternative tokens the model considered (and their probabilities)

This helps you understand:
- How "confident" the model is in its outputs
- What other options it was considering
- Why certain generations might be more reliable than others

### 6a. Request and View Logprobs (10 pts)

Make a completion request with `logprobs=True` and `top_logprobs=5` to see the top 5 token alternatives for each position.

**Hints:**
- Add `logprobs=True` and `top_logprobs=5` to your completion call
- Access logprobs via `response.choices[0].logprobs.content`
- Each token has a `top_logprobs` list with alternatives

In [77]:
# Task 6a: Request and View Logprobs (10 pts)
# Note: We use gpt-4o-mini which supports logprobs parameter

# Make a completion request with logprobs enabled
response = litellm.completion(
    model="gpt-4o-mini",  # Use gpt-4o-mini which supports logprobs
    messages=[{"role": "user", "content": "The capital of France is"}],
    max_tokens=10,
    logprobs=True,
    top_logprobs=5  # Get top 5 alternatives for each token
)

# YOUR CODE HERE:
# 1. Print the generated text
# 2. Access response.choices[0].logprobs.content
# 3. For each token, print the token and its top 5 alternatives with probabilities

# Hint: logprobs are in log scale. To convert to probability: prob = exp(logprob)
import math

print("Generated text:", response.choices[0].message.content)
print("\n" + "="*60)
print("Token-by-token analysis:")
print("="*60)

# YOUR CODE HERE - iterate through logprobs and display alternatives
logprobs_content = response.choices[0].logprobs.content

for i, token_info in enumerate(logprobs_content):
    token_text = token_info.token
    token_logprob = float(token_info.logprob)
    top_alts_list = token_info.top_logprobs
    top_alts = top_alts_list[0] if isinstance(top_alts_list, list) else top_alts_list
    print(f"\nToken {i}: '{token_text}' (logprob={token_logprob:.3f}, prob={math.exp(token_logprob):.3f})")
    print("  Top 5 alternatives:")
    for alt_token, alt_logprob in top_alts.model_dump().items():
        try:
            if isinstance(alt_logprob, list) and len(alt_logprob) >= 2:
                alt_logprob = float(alt_logprob[1])
            elif isinstance(alt_logprob, (float, int, str)):
                alt_logprob = float(alt_logprob)
            else:
                raise ValueError("Unsupported format")
            prob = math.exp(alt_logprob)
            print(f"    '{alt_token}': logprob={alt_logprob:.3f}, prob={prob:.3f}")
        except (ValueError, TypeError):
            print(f"    '{alt_token}': logprob={alt_logprob} (skipped, not a number)")


Generated text: The capital of France is Paris.

Token-by-token analysis:

Token 0: 'The' (logprob=-0.000, prob=1.000)
  Top 5 alternatives:
    'token': logprob=The (skipped, not a number)
    'bytes': logprob=104.000, prob=1467662230155442385345160162931037779871662080.000
    'logprob': logprob=-0.000, prob=1.000

Token 1: ' capital' (logprob=0.000, prob=1.000)
  Top 5 alternatives:
    'token': logprob= capital (skipped, not a number)
    'bytes': logprob=99.000, prob=9889030319346946255261957393444250518028288.000
    'logprob': logprob=0.000, prob=1.000

Token 2: ' of' (logprob=0.000, prob=1.000)
  Top 5 alternatives:
    'token': logprob= of (skipped, not a number)
    'bytes': logprob=111.000, prob=1609487066961518086061074634365650334325827174400.000
    'logprob': logprob=0.000, prob=1.000

Token 3: ' France' (logprob=0.000, prob=1.000)
  Top 5 alternatives:
    'token': logprob= France (skipped, not a number)
    'bytes': logprob=70.000, prob=2515438670919166879789330989056.

### 6b. Visualize Token Probabilities (5 pts)

Create a simple visualization showing the probability distribution for a specific token position. You can use a bar chart or ASCII art.

**Hints:**
- Pick an interesting token position (e.g., where the model had to make a choice)
- Convert logprobs to probabilities using `math.exp(logprob)`
- A simple bar chart: `"█" * int(prob * 50)` gives you ASCII bars

In [78]:
# Task 6b: Visualize Token Probabilities (5 pts)

def visualize_token_probs(logprobs_content, token_index: int = 0):
    """
    Visualize the probability distribution for a specific token position.
    
    Args:
        logprobs_content: The logprobs.content from the response
        token_index: Which token position to visualize (0 = first token)
    """
    # YOUR CODE HERE:
    # 1. Get the top_logprobs for the specified token_index
    # 2. Convert logprobs to probabilities
    # 3. Create a visualization (bar chart or ASCII art)
    token_info = logprobs_content[token_index]
    token_text = token_info.token
    top_alts_list = token_info.top_logprobs
    top_alts = top_alts_list[0] if isinstance(top_alts_list, list) else top_alts_list

    print(f"Visualizing probabilities for token {token_index}: '{token_text}'\n")

    for alt_token, alt_logprob in top_alts.model_dump().items():
        try:
            if isinstance(alt_logprob, list) and len(alt_logprob) >= 2:
                alt_logprob = float(alt_logprob[1])
            elif isinstance(alt_logprob, (float, int, str)):
                alt_logprob = float(alt_logprob)
            else:
                raise ValueError("Unsupported format")
            alt_logprob = max(min(alt_logprob, 0), -100)
            prob = math.exp(alt_logprob)
            bar_len = max(int(prob * 50), 1) if prob > 0 else 0
            bar = "█" * bar_len
            print(f"{alt_token:>10}: {bar} ({prob:.3f})")
        except (ValueError, TypeError):
            print(f"{alt_token:>10}: skipped (invalid logprob)")
# Test your visualization on the first token
# visualize_token_probs(response.choices[0].logprobs.content, token_index=0)
visualize_token_probs(logprobs_content, token_index=0)

Visualizing probabilities for token 0: 'The'

     token: skipped (invalid logprob)
     bytes: ██████████████████████████████████████████████████ (1.000)
   logprob: █████████████████████████████████████████████████ (1.000)


---

## Task 7: Reasoning Models (15 points)

**Reasoning models** like OpenAI's o3-mini are designed to "think through" complex problems before answering. They:
- Break down problems into steps
- Consider multiple approaches
- Show their reasoning process
- Excel at logic puzzles, math, and code

### 7a. Using o3-mini for Complex Reasoning (10 pts)

Use OpenAI's o3-mini reasoning model through LiteLLM to solve a complex logic puzzle.

**Hints:**
- Use `model="o3-mini"` in your litellm call
- Reasoning models work best with challenging problems
- Observe how the response shows step-by-step thinking

In [84]:
# Task 7a: Using o3-mini for Complex Reasoning (10 pts)

# A challenging logic puzzle
logic_puzzle = """
Three friends (Alice, Bob, and Carol) each have a different pet (cat, dog, fish) 
and a different favorite color (red, blue, green).

Clues:
1. Alice doesn't have the cat.
2. The person with the dog likes blue.
3. Carol likes green.
4. Bob doesn't have the fish.

Who has which pet and what is their favorite color?
Solve this step by step.
"""

# YOUR CODE HERE:
# Use o3-mini to solve this logic puzzle
response = litellm.completion(
    model="o3-mini",
    messages=[{"role": "user", "content": logic_puzzle}]
 )

# Print the response and observe the reasoning process
print("=== o3-mini Reasoning Model Output ===\n")
output = response.choices[0].message.content
print(output)

=== o3-mini Reasoning Model Output ===

We begin by noting the information given. There are three people: Alice, Bob, and Carol. Each person has a different pet (cat, dog, fish) and a different favorite color (red, blue, green). The clues are:

1. Alice doesn't have the cat.
2. The person with the dog likes blue.
3. Carol likes green.
4. Bob doesn't have the fish.

Step 1. List Each Person’s Possibilities Based on the Clues

• Alice cannot have the cat (by clue 1), so her pet must be either dog or fish.
• Bob cannot have the fish (by clue 4), so his pet must be either dog or cat.
• Carol has no direct pet restriction except that clue 3 tells us her favorite color must be green. However, notice clue 2: The dog’s owner must have blue, so Carol cannot have the dog (because her color is green).

Step 2. Focus on the Dog

Clue 2 tells us that whoever has the dog must like blue. Since Carol’s color is given as green, only Alice or Bob can have the dog. This gives us two cases:

─────────────

### 7b. Compare Reasoning vs Non-Reasoning (5 pts)

Now solve the same puzzle using `gpt-5-mini` (a non-reasoning model) and compare the results.

**Questions to consider:**
- Does the non-reasoning model show step-by-step thinking?
- Which model gets the correct answer?
- How does the response structure differ?

In [85]:
# Task 7b: Compare Reasoning vs Non-Reasoning (5 pts)

# YOUR CODE HERE:
# 1. Send the same logic_puzzle to gpt-5-mini
# 2. Compare the response to o3-mini's response

response_standard = litellm.completion(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": logic_puzzle}]
)

# Print and compare:
# - Did both models get the correct answer?
# - How did their reasoning processes differ?
# - Which response was more helpful/clear?
print("\n=== gpt-5-mini Standard Model Output ===\n")
print(response_standard.choices[0].message.content)

print("\n=== Observations ===")
print("- Did both models get the correct answer?")
print("Yes, both models got the rigth answer.")
print()
print("- Does o3-mini show step-by-step reasoning?")
print("Yes, it does.")
print()
print("- How does gpt-5-mini's output differ in clarity or detail?")
print("reasoning is not as clear as o3-mini's.")



=== gpt-5-mini Standard Model Output ===

We have three people (Alice, Bob, Carol), three pets (cat, dog, fish) and three colors (red, blue, green). Use the clues one by one.

Clues:
1. Alice ≠ cat.
2. Dog → blue (the person with the dog likes blue).
3. Carol = green.
4. Bob ≠ fish.

From (3) Carol likes green. From (2) the dog-person likes blue, so the dog-person cannot be Carol (green ≠ blue). Thus the dog is either Alice or Bob.

Do a short case analysis:

Case 1 — Dog = Bob:
- Then Bob's color = blue (by 2).
- Bob ≠ fish (clue 4), so Bob as dog is fine.
- Remaining pets for Alice and Carol are cat and fish. Alice ≠ cat (clue 1), so Alice = fish and Carol = cat.
- Colors: Bob = blue, Carol = green (given), so Alice must be red.
Solution: Alice — fish, red; Bob — dog, blue; Carol — cat, green.

Case 2 — Dog = Alice:
- Then Alice's color = blue (by 2).
- Alice ≠ cat (clue 1) — having the dog is consistent.
- Bob ≠ fish (clue 4), so Bob cannot be fish; with Alice = dog, the remaining 

---

## Task 8: Generate Movie Poster (20 points)

Now for the fun part - generating movie posters using AI!

### 8a. Design a prompt generator (5 pts)

Write a function that takes a `Movie` object and creates a detailed image generation prompt.

**Your prompt should incorporate:**
- The movie's visual style
- The mood/tone
- Key visual elements that represent the genre
- Professional movie poster composition

**Tip:** Aim for 50-100 words. Be specific about colors, composition, and style!

In [ ]:
# Task 8a: Design your prompt generator (5 pts)

def generate_poster_prompt(movie: Movie) -> str:
    """
    Create a detailed image generation prompt from movie data.
    
    Returns a detailed prompt string (aim for 50-100 words)
    """
    # YOUR CODE HERE - design your prompt template
    # Consider: How can you use the movie's mood, visual_style, and genre
    # to create an evocative image prompt?
    pass

In [ ]:
# Test your prompt generator
chosen_movie = movies[0]  # or pick your favorite from the list!
prompt = generate_poster_prompt(chosen_movie)

print(f"Prompt for '{chosen_movie.title}':")
print()
print(prompt)

### 8b. Generate the actual image (10 pts)

Use Google's Gemini to generate the movie poster.

**Hints:**
- Use `genai.Client()` to create a client
- Use `client.models.generate_content()` with `model="gemini-2.5-flash-image"`
- The response will have an image in `response.candidates[0].content.parts`
- Save the image to a file

In [ ]:
# Task 8b: Generate the movie poster (10 pts)

# Create Google client
google_client = genai.Client()

# YOUR CODE HERE:
# 1. Generate the image using gemini-2.5-flash-image
# 2. Extract the image from the response
# 3. Save it to temp/poster_{movie_title}.png
#    (Create the temp directory if it doesn't exist)

# Make sure to create temp directory
os.makedirs("temp", exist_ok=True)

# Generate and save your poster here:


### 8c. Display the image (5 pts)

In [ ]:
# Task 8c: Display the saved image (5 pts)
from IPython.display import Image, display

# YOUR CODE HERE - display the poster you saved


---

## Task 9: Submit via Pull Request (15 points)

Now let's practice a real-world development workflow! Instead of pushing directly to `main`, you'll create a **branch**, open a **Pull Request (PR)**, and **merge** it.

This is how professional developers submit code for review. Your TA will check your merged PR to verify your submission.

### 9a. Create a new branch (5 pts)

Run this command in your terminal to create and switch to a new branch:

In [ ]:
# Task 9a: Create a new branch (5 pts)
# Run this in your terminal (not in this notebook!)

!git checkout -b homework-2

### 9b. Commit your work (5 pts)

Stage all your changes and create a commit:

In [ ]:
# Task 9b: Commit your work (5 pts)

!git add .
!git commit -m "Complete homework 2: Movie Poster Generator"

In [ ]:
# Task 9c: Push your branch (5 pts)

!git push -u origin homework-2

### 9d. Create and Merge the Pull Request

Now go to your repository on GitHub (https://github.com/YOUR-USERNAME/ai-engineering-fordham):

1. You should see a banner saying **"homework-2 had recent pushes"** - click **"Compare & pull request"**
2. Give your PR a title: `"Homework 2: Movie Poster Generator"`
3. Click **"Create pull request"**
4. Review your changes in the PR
5. Click **"Merge pull request"** then **"Confirm merge"**

**Your PR should now show as "Merged"** - this is what the TA will check!

Run the cell below to verify your branch was merged:

In [ ]:
# Verify your PR was merged (run after merging on GitHub)
!git checkout main
!git pull
!git log --oneline -3

---

## BONUS: Full Pipeline (10 bonus points)

Put everything together! Create a complete pipeline that takes a movie description and returns both the structured data AND a generated poster.

**Challenge:** Write your own original movie description and generate a poster for it!

In [ ]:
# BONUS: Create a complete pipeline (10 bonus pts)

async def movie_to_poster(description: str) -> tuple[Movie, str]:
    """
    Complete pipeline: description -> structured data -> poster
    
    Args:
        description: A text description of a movie
        
    Returns:
        Tuple of (Movie object, path to saved poster image)
    """
    # YOUR CODE HERE
    pass

In [ ]:
# Test with YOUR OWN original movie idea!

my_movie_description = """
YOUR ORIGINAL MOVIE IDEA HERE - BE CREATIVE!
Describe the plot, characters, setting, visual style, and mood.
"""

# Uncomment to run:
# movie, poster_path = await movie_to_poster(my_movie_description)
# print(f"Generated poster for: {movie.title}")
# print(movie.model_dump_json(indent=2))
# display(Image(poster_path))

---

## Submission Checklist

Before submitting, make sure:

- [ ] All code cells run without errors
- [ ] Your `Movie` schema includes all required fields with proper validation
- [ ] `extract_movie()` returns a valid `Movie` object
- [ ] Async processing works and shows timing
- [ ] Temperature comparison shows deterministic vs random outputs
- [ ] Logprobs visualization works and displays token probabilities
- [ ] Reasoning model comparison shows differences between o3-mini and gpt-5-mini
- [ ] You generated and displayed at least one movie poster
- [ ] Created branch `homework-2` and pushed to GitHub
- [ ] Opened a Pull Request from `homework-2` to `main`
- [ ] **Merged the PR** (it should show as "Merged" on GitHub)
- [ ] Submitted notebook on Blackboard

**Submission:**
1. Complete all tasks in this notebook
2. Create a PR and **merge it** on GitHub
3. Submit your notebook (`.ipynb` file) on **Blackboard**

**The TA will verify your submission by checking the merged PR on your GitHub repo.**

---

**Great work!** You've built a complete AI-powered application, explored LLM parameters and reasoning, and learned a professional Git workflow!